# Speech Transcription on IPUs using Whisper - Inference

This notebook demonstrates speech transcription on the IPU using the [Whisper implementation in the Hugging Face Transformers library](https://huggingface.co/spaces/openai/whisper) alongside [Optimum Graphcore](https://github.com/huggingface/optimum-graphcore).

Whisper is a versatile speech recognition model that can transcribe speech as well as perform multi-lingual translation and recognition tasks.
It was trained on diverse datasets to give human-level speech recognition performance without the need for fine-tuning. 

[🤗 Optimum Graphcore](https://github.com/huggingface/optimum-graphcore) is the interface between the [🤗 Transformers library](https://huggingface.co/docs/transformers/index) and [Graphcore IPUs](https://www.graphcore.ai/products/ipu).
It provides a set of tools enabling model parallelization and loading on IPUs, training and fine-tuning on all the tasks already supported by 🤗 Transformers while being compatible with the 🤗 Hub and every model available on it out of the box.

> **Hardware requirements:** The Whisper models `whisper-tiny`, `whisper-base` and `whisper-small` can run two replicas on the smallest IPU-POD4 machine. The most capable model, `whisper-large`, will need to use either an IPU-POD16 or a Bow Pod16 machine. Please contact Graphcore if you'd like assistance running model sizes that don't work in this simple example notebook.

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you.

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

## Dependencies

In order to improve usability and support for future users, Graphcore would like to collect information about the
applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext graphcore_cloud_tools.notebook_logging.gc_logger` from any cell.

Install the dependencies the notebook needs.

In [ ]:
# Install optimum from source 
!pip install git+https://github.com/huggingface/optimum-graphcore.git@1f13c9279921bd064a0a857b044d9c18f7fbca13 "tokenizers<0.13" "transformers==4.25.1" "soundfile" "librosa" "matplotlib"
%pip install "graphcore-cloud-tools[logger] @ git+https://github.com/graphcore/graphcore-cloud-tools"
%load_ext graphcore_cloud_tools.notebook_logging.gc_logger

## Running Whisper on the IPU

We start by importing the required modules, some of which are needed to configure the IPU.


In [ ]:
# Generic imports
import os
from datasets import load_dataset
import matplotlib.pyplot as plt
import librosa
import IPython
import random


# IPU-specific imports
from optimum.graphcore import IPUConfig
from optimum.graphcore.modeling_utils import to_pipelined
from optimum.graphcore.models.whisper import WhisperProcessorTorch

# HF-related imports
from transformers import WhisperForConditionalGeneration

This notebook demonstrates how to run all sizes of Whisper, assuming you meet the IPU hardware requirements:

- `whisper-tiny`, `base` and `small` only requires 2 IPUs (IPU-POD4)
- `whisper-medium` requires 4 IPUs (IPU-POD4)
- `whisper-large` requires 8 IPUs (IPU-POD16 or a Bow Pod16)

The Whisper model is available on Hugging Face in several sizes, from `whisper-tiny` with 39M parameters to `whisper-large` with 1550M parameters.

The [Whisper architecture](https://openai.com/research/whisper) is an encoder-decoder Transformer, with the audio split into 30-second chunks.
- For `whisper-tiny`, `small` and `base`, one IPU is used for the encoder part of the graph and another for the decoder part.
- For `whisper-medium `, two IPUs are used to place the encoder part and two others for the decoder part.
- For `whisper-large `, four IPUs are used to place the encoder part and four others for the decoder part.

The `IPUConfig` object helps to configure the model to be pipelined across the IPUs.
The number of transformer layers per IPU can be adjusted by using `layers_per_ipu`.

In [ ]:
num_available_ipus=int(os.getenv("NUM_AVAILABLE_IPU", 4))
cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "./exe_cache") + "/whisper"

default_ipu_config = IPUConfig(executable_cache_dir=cache_dir,
                               ipus_per_replica=2)

medium_ipu_config = IPUConfig(executable_cache_dir=cache_dir,
                             ipus_per_replica=4,
                             layers_per_ipu=[12, 12, 13, 11])

large_ipu_config = IPUConfig(executable_cache_dir=cache_dir,
                             ipus_per_replica=8,
                             layers_per_ipu=[8, 8, 8, 8, 6, 9, 9, 8])

configs = {
    "tiny": ("openai/whisper-tiny.en", 
        default_ipu_config),
    
    "base": ("openai/whisper-base.en", 
        default_ipu_config),

    "small": ("openai/whisper-small.en", 
        default_ipu_config),
    
    "medium": ("openai/whisper-medium.en",
        medium_ipu_config),

    "large": ("openai/whisper-large-v2", 
        large_ipu_config),
}


def select_whisper_config(size: str, custom_checkpoint: str):
    auto_sizes = {4: "tiny", 16: "large"}
    if size == "auto":
        selected_size = auto_sizes[num_available_ipus]
    elif size in configs.keys():
        if size == "large" and num_available_ipus < 8:
            raise ValueError("Error: You need at least 8 IPUs to run whisper-large "
                             f"but your current environment has {num_available_ipus} IPUs available.")
        selected_size = size
    else:
        raise ValueError(f"{size} is not a valid size for Whisper")
    
    model_checkpoint, ipu_config = configs[selected_size]
    if custom_checkpoint is not None:
        model_checkpoint = custom_checkpoint

    print(f"Using whisper-{selected_size} config with the checkpoint '{model_checkpoint}'.")
    return model_checkpoint, ipu_config 

Select the Whisper size bellow, try `"tiny"`,`"base"`, `"small"`, `"medium"`, `"large"`  or let the `"auto"` mode choose for you.

In [ ]:
model_checkpoint, ipu_config = select_whisper_config("auto", custom_checkpoint=None) 

You can also use a custom checkpoint from Hugging Face Hub using the argument `custom_checkpoint` above. In this case, you have to make sure that `size` matches the checkpoint model size.

In [ ]:
# Instantiate processor and model
processor = WhisperProcessorTorch.from_pretrained(model_checkpoint)
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)

# Adapt whisper-tiny to run on the IPU

pipelined_model = to_pipelined(model, ipu_config)
pipelined_model = pipelined_model.parallelize(
    for_generation=True, 
    use_cache=True, 
    batch_size=1, 
    max_length=448,
    on_device_generation_steps=16, 
    use_encoder_output_buffer=True).half()

Now we can load the dataset and process an example audio file.
If precompiled models are not available, then the first run of the model triggers two graph compilations.
This means that our first test transcription could take a minute or two to run, but subsequent runs will be much faster.

In [ ]:
# load the dataset and read an example sound file
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
test_sample = ds[2]
sample_rate = test_sample['audio']['sampling_rate']

def transcribe(data, rate):
    input_features = processor(data, return_tensors="pt", sampling_rate=rate).input_features.half()

    # This triggers a compilation, unless a precompiled model is available.
    sample_output = pipelined_model.generate(
        input_features,
        use_cache=True,
        do_sample=False,
        max_length=448, 
        min_length=3)
    transcription = processor.batch_decode(sample_output, skip_special_tokens=True)[0]
    return transcription

test_transcription = transcribe(test_sample["audio"]["array"], sample_rate)

In the next cell, we compare the expected text from the dataset with the transcribed result from the model.
There will typically be some small differences, but even `whisper-tiny` does a great job! It even adds punctuation.

You can listen to the audio and compare the model result yourself using the controls below.

In [ ]:
print(f"Expected: {test_sample['text']}\n")
print(f"Transcribed: {test_transcription}")

plt.figure(figsize=(14, 5))
librosa.display.waveshow(test_sample["audio"]["array"], sr=sample_rate)
IPython.display.Audio(test_sample["audio"]["array"], rate=sample_rate)

The model only needs to be compiled once. Subsequent inferences will be much faster.
In the cell below, we repeat the exercise but with a random example from the dataset.

You might like to re-run this next cell multiple times to get different comparisons.

In [ ]:
idx = random.randint(0, ds.num_rows - 1)
data = ds[idx]["audio"]["array"]

print(f"Example #{idx}\n")
print(f"Expected: {ds[idx]['text']}\n")
print(f"Transcribed: {transcribe(data, sample_rate)}")

plt.figure(figsize=(14, 5))
librosa.display.waveshow(data, sr=sample_rate)
IPython.display.Audio(data, rate=sample_rate, autoplay=True)

Finally, we detach the process from the IPUs when we are done to make the IPUs available to other users.

In [ ]:
pipelined_model.detachFromDevice()

## Next Steps

The `whisper-tiny` model used here is very fast for inference and so cheap to run, but its accuracy can be improved.
The `whisper-base`, `whisper-small` and `whisper-medium` models have 74M, 244M and 769 M parameters respectively (compared to just 39M for `whisper-tiny`). You can try out `whisper-base`, `whisper-small` and `whisper-medium` by changing `select_whisper_config("auto")` (at the beginning of this notebook) to:
- `select_whisper_config("base")`
- `select_whisper_config("small")`
- `select_whisper_config("medium")` respectively.

Larger models and multilingual models are also available.
To access the multilingual models, remove the `.en` from the checkpoint name. Note however that the multilingual models are slightly less accurate for this English transcription task but they can be used for transcribing other languages or for translating to English.

The largest model `whisper-large` has 1550M parameters and requires a 8-IPUs pipeline.
You can try it by setting `select_whisper_config("large")`
To run it you will need more than the IPU-POD4. On Paperspace, this is available using either an IPU-POD16 or a Bow Pod16 machine. Please contact Graphcore if you need assistance running these larger models.


## Conclusion

In this notebook, we demonstrated using Whisper for speech recognition and transcription on the IPU.
We used the Optimum Graphcore package to interface between the IPU and the Hugging Face Transformers library. This meant that only a few lines of code were needed to get this state-of-the-art automated speech recognition model running on IPUs.